# Setting

In [ ]:
from pf_utils import PortfolioManager as PM 
from pf_utils import BayesianEstimator, performance_stats, get_date_range, METRICS

In [ ]:
path_data = 'data'
path_tran = 'transaction'

In [ ]:
file_cost='transaction_cost'

# Data

In [ ]:
universe = 'UV_FUND'
fund = PM.create_universe(universe)
cost = PM.get_cost(universe, file=file_cost, path=path_tran)
fee = cost['fee']

In [ ]:
df_prices = fund.df_prices
df_prices_fees = fund._get_prices_after_fee(df_prices, fee)

In [ ]:
days_in_year = fund.days_in_year
df_year = get_date_range(df_prices).loc[:, 'n'].div(days_in_year).rename('year')
_ = df_year.groupby(df_year.round()).count().cumsum().div(df_year.count()).plot(title='Cumulative distribution of data size')

# Bayesian

*revert data to original montly for estimation*

In [ ]:
days_in_year = 12
df_prc = df_prices_fees.resample('M').last()

## 1 Year Rate of Return

In [ ]:
freq_year = 1
min_year = 4 # min data size

In [ ]:
file_est = f'fund_f{freq_year}y_s{min_year}y_250109'
file_est

### Split

In [ ]:
tickers = df_year.loc[df_year >= min_year].index.to_list()
len(tickers)

In [ ]:
n = freq_year * days_in_year
df_p = df_prc[tickers]
df_train = df_p.iloc[:-n]
df_val = df_p.iloc[-n:]

In [ ]:
len(df_p), len(df_train), len(df_val)

### Estimate

In [ ]:
kw = dict(
    days_in_year = days_in_year,
    security_names = fund.security_names
)

*Sample*

In [ ]:
freq = freq_year * days_in_year
be = BayesianEstimator(df_train, **kw)
be.bayesian_sample(freq=freq, file=file_est, path=path_data)

*Load*

In [ ]:
be = BayesianEstimator.create(file_est, path_data, **kw)

#### Visualize

In [ ]:
#xlims = None
xlims = [(-0.3, 0.4), (-1, 2.5)]
_ = be.plot_returns(max_legend=10, xlims=xlims)

### Compare

In [ ]:
#var_name = 'cagr'
var_name = 'total_return' # freq 1y
ms = ['mean','sd','hdi_3%','hdi_97%']
ascending = [False, True, False, False]

df_s = be.bayesian_summary(var_name).droplevel(0)
pick = lambda i, n=5, df=df_s: df.sort_values(ms[i], ascending=ascending[i]).iloc[:n].index.to_list()

In [ ]:
#df_s.isna().any(axis=1).sum()
#df_s = df_s.dropna()

In [ ]:
t0 = pick(0)
kw = dict(
    ref_val='default', 
    figsize=(12,2.5), textsize=9, grid=(1,5), length=16)
be.plot_posterior(var_name, t0, **kw)

In [ ]:
t1 = pick(1)
be.plot_posterior(var_name, t1, **kw)

In [ ]:
t2 = pick(2)
be.plot_posterior(var_name, t2, **kw)

In [ ]:
t3 = pick(3)
be.plot_posterior(var_name, t3, **kw)

### Validate

In [ ]:
m = var_name
start = df_val.index.min()
fund.plot(t0, fee=fee, base=1000, metric=m, start_date=start)

In [ ]:
fund.plot(t2, fee=fee, base=1000, metric=m, start_date=start)

## 3 Year Rate of Return

In [ ]:
freq_year = 3

### Min years 7

In [ ]:
min_year = 7 # min data size

In [ ]:
file_est = f'fund_f{freq_year}y_s{min_year}y_250109'
file_est

#### Split

In [ ]:
tickers = df_year.loc[df_year >= min_year].index.to_list()
len(tickers)

In [ ]:
n = freq_year * days_in_year
df_p = df_prc[tickers]
df_train = df_p.iloc[:-n]
df_val = df_p.iloc[-n:]

In [ ]:
len(df_p), len(df_train), len(df_val)

#### Estimate

In [ ]:
kw = dict(
    days_in_year = days_in_year,
    security_names = fund.security_names
)

*Sample*

In [ ]:
freq = freq_year * days_in_year
be = BayesianEstimator(df_train, **kw)
be.bayesian_sample(freq=freq, file=file_est, path=path_data)

*Load*

In [ ]:
be = BayesianEstimator.create(file_est, path_data, **kw)

##### Visualize

In [ ]:
#xlims = None
xlims = [(-0.5, 1.5), (-0.5, 6)]
_ = be.plot_returns(max_legend=10, xlims=xlims)

#### Compare

In [ ]:
#var_name = 'cagr'
var_name = 'total_return'
ms = ['mean','sd','hdi_3%','hdi_97%']
ascending = [False, True, False, False]

df_s = be.bayesian_summary(var_name).droplevel(0)
pick = lambda i, n=5, df=df_s: df.sort_values(ms[i], ascending=ascending[i]).iloc[:n].index.to_list()

In [ ]:
#df_s.isna().any(axis=1).sum()
#df_s = df_s.dropna()

In [ ]:
t0 = pick(0)
kw = dict(
    ref_val='default', 
    figsize=(12,2.5), textsize=9, grid=(1,5), length=16)
be.plot_posterior(var_name, t0, **kw)

In [ ]:
t2 = pick(2)
be.plot_posterior(var_name, t2, **kw)

In [ ]:
t3 = pick(3)
be.plot_posterior(var_name, t3, **kw)

#### Validate

In [ ]:
m = var_name
start = df_val.index.min()
fund.plot(t2, fee=fee, base=1000, metric=m, start_date=start)

In [ ]:
fund.plot(t0, fee=fee, base=1000, metric=m, start_date=start)

### Min years 3 w/o Validation

In [ ]:
min_year = 3 # min data size

In [ ]:
file_est = f'fund_f{freq_year}y_s{min_year}y_250109'
file_est

#### Split

In [ ]:
tickers = df_year.loc[df_year >= min_year].index.to_list()
len(tickers)

*no validation*

In [ ]:
df_p = df_prc[tickers]
df_train = df_p

In [ ]:
len(df_train)

#### Estimate

In [ ]:
kw = dict(
    days_in_year = days_in_year,
    security_names = fund.security_names
)

*Sample*

In [ ]:
freq = freq_year * days_in_year
be = BayesianEstimator(df_train, **kw)
# uncomment for new sampling 
#be.bayesian_sample(freq=freq, file=file_est, path=path_data)

*Load*

In [ ]:
be = BayesianEstimator.create(file_est, path_data, **kw)

##### Visualize

In [ ]:
#xlims = None
xlims = [(-0.3, 0.8), (-4, 10)]
_ = be.plot_returns(max_legend=10, xlims=xlims)

#### Compare

In [ ]:
var_name = 'total_return'
ms = ['mean','sd','hdi_3%','hdi_97%']
ascending = [False, True, False, False]

df_s = be.bayesian_summary(var_name).droplevel(0)
pick = lambda i, n=5: df_s.sort_values(ms[i], ascending=ascending[i]).iloc[:n].index.to_list()

In [ ]:
t0 = pick(0)
kw = dict(
    plotly=False,
    ref_val='default', 
    figsize=(12,3), textsize=9, grid=(1,5), length=16)
be.plot_posterior(var_name, t0, **kw)

In [ ]:
be.plot_posterior(var_name, t0, error=0.9999)

In [ ]:
t2 = pick(2)
be.plot_posterior(var_name, t2)

In [ ]:
t3 = pick(3)
be.plot_posterior(var_name, t3)

*choice from frequentist stats*

In [ ]:
n = 5
t = be.get_stats(var_name).T.sort_values(var_name, ascending=False).iloc[:n].index.to_list()
be.plot_posterior(var_name, t)

#### Cases

##### TDF

In [ ]:
from math import ceil
tdf = 'tdf2030'
t9 = pick(2, 99)
t9 = [x for x in t9 if tdf in fund.security_names[x].lower()]
grid = (ceil(len(t9)/5), 5)
#be.plot_posterior(var_name, t9, **{**kw, 'grid':grid, 'figsize':(12,3*grid[0])})
be.plot_posterior(var_name, t9)

##### Asia

In [ ]:
t9 = ['K55364CF7048','K55303BT4176','K55234BY9966','K55235BW6799']
grid = (1,4)
be.plot_posterior(var_name, t9, **{**kw, 'grid':grid, 'figsize':(12,3*grid[0])})

In [ ]:
fund.get_names(t9)

# Visualize

In [1]:
from pf_utils import PortfolioManager as PM 
from pf_dash import create_app, add_density_plot

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
path_data = 'data'
path_tran = 'transaction'
file_cost = 'transaction_cost'

In [3]:
universe = 'UV_FUND'
fund = PM.create_universe(universe)
cost = PM.get_cost(universe, file=file_cost, path=path_tran)
fee = cost['fee']

64 securities from 2008-11-28 to 2024-12-31 uploaded.
REMINDER: 64 equities converted to daily (days in year: 365)
Daily metrics in Performance statistics must be meaningless
Price data loaded
Cost data transaction_cost_241217.csv loaded


In [4]:
df_prices = fund.df_prices
df_prices_fees = fund._get_prices_after_fee(df_prices, fee)

In [5]:
df_prc = df_prices.resample('M').last()
df_prc_f = df_prices_fees.resample('M').last()
fund_name = fund.security_names

In [6]:
tickers = None
#tickers = ['K55364CF7048','K55303BT4176','K55234BY9966','K55235BW6799']
app = create_app(df_prc, df_prc_f, fund_name=fund_name, tickers=tickers)

In [22]:
from pf_dash import get_inference, update_inference_data, update_inference_plot
from dash import Dash, html, dcc, Output, Input
import dash_bootstrap_components as dbc


def add_density_plot(app, file=None, path=None, tickers=None, fund_name=None,
                     n_points=500, error=0.999, option_all='All', n_default=5):
    data_inf = get_inference(file, path, tickers=tickers, n_points=n_points, error=error)

    # update layout of the app
    new_tab = dbc.Tab(dcc.Graph(id='density-plot'), label='추정')

    # Locate the Row containing Tabs and append the new Tab
    app.add_tab(new_tab)

    # Add density-data Store to the layout
    app.layout.children.append(
        dcc.Store(id='density-data')
    )

    @app.callback(
        Output('density-data', 'data'),
        Input('ticker-dropdown', 'value')
    )
    def _update_inference_data(tickers):
        """
        process data and save to dcc.Store
        """
        return update_inference_data(tickers, data_inf, option_all=option_all, n_default=n_default)
        
    
    @app.callback(
        Output('density-plot', 'figure'),
        Input('density-data', 'data')
    )
    def _update_inference_plot(data):
        return update_inference_plot(data, fund_name)

In [7]:
kw_dst = dict(
    file = 'fund_f3y_s3y_250109.pkl',
    path = 'data',
    tickers = tickers,
    fund_name = fund_name
)
add_density_plot(app, **kw_dst)

In [8]:
app.run_server()

In [9]:
add_density_plot(app, **kw_dst)

ERROR: tab '추정' already exits


In [29]:
new_tab = dbc.Tab(dcc.Graph(id='density-plot'), label='추정2')

for row in app.layout.children:
    if isinstance(row, dbc.Row):
        if isinstance(row.children, dbc.Tabs):
            labels = [x.label for x in row.children.children]
            if new_tab.label in labels:
                print(f"ERROR: tab '{new_tab.label}' already exits")
            else:
                row.children.children.append(new_tab)
                break

In [30]:
row.children.children

[Tab(children=Graph(id='price-plot'), label='가격'),